In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

X_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
X_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
X_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

Epoch 1, Training Loss: 0.0123, Validation Loss: 0.0009, Validation Accuracy: 0.9999
Epoch 2, Training Loss: 0.0004, Validation Loss: 0.0008, Validation Accuracy: 0.9999
Epoch 3, Training Loss: 0.0004, Validation Loss: 0.0005, Validation Accuracy: 0.9999
Epoch 4, Training Loss: 0.0003, Validation Loss: 0.0002, Validation Accuracy: 0.9999
Epoch 5, Training Loss: 0.0002, Validation Loss: 0.0002, Validation Accuracy: 0.9999
Epoch 6, Training Loss: 0.0001, Validation Loss: 0.0016, Validation Accuracy: 0.9998
Early stopping triggered


In [ ]:
input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))


dnn_model = Sequential()
dnn_model.add(Input(shape=input_shape))
dnn_model.add(Dense(units=30, activation='relu'))
dnn_model.add(Dense(units=20, activation='relu'))
dnn_model.add(Dense(units=num_classes, activation='softmax'))

opt = SGD(learning_rate=0.01)

dnn_model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
hist = dnn_model.fit(X_train, y_train, epochs=15, batch_size=64,
                     validation_data=(X_val, y_val),
                     callbacks=[early_stopping])

# test_loss, test_acc = dnn_model.evaluate(x_test, y_test)
# print('Test accuracy:', test_acc)

# y_hat = dnn_model.predict(x_test)
# y_hat = np.argmax(y_hat, axis=-1)

2024-06-04 21:16:48.859665: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78211 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:10:00.0, compute capability: 8.0


Epoch 1/15


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import torch

import time

from a2pm import A2PMethod
from a2pm.callbacks import BaseCallback, MetricCallback, TimeCallback
from a2pm.patterns import BasePattern, CombinationPattern, IntervalPattern
from a2pm.wrappers import BaseWrapper, KerasWrapper, SklearnWrapper, TorchWrapper

In [1]:


def calculate_performance_metrics(model, X_test, y_test, type):
    metrics_weighted(model, X_test, y_test, type)
    metrics_macro(model, X_test, y_test, type)

def metrics_macro(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=-1)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    data = {
    "Metric": ["Accuracy", "Macro Precision", "Macro Recall", "Macro F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/DNN_tensor.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Macro Precision: {precision * 100:.2f}%")
    print(f"Macro Recall: {recall * 100:.2f}%")
    print(f"Macro F1 Score: {f1 * 100:.2f}%")
    
def metrics_weighted(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=-1)

    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    data = {
    "Metric": ["Accuracy", "Weighted Precision", "Weighted Recall", "Weighted F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/DNN_tensor.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Weighted Precision: {precision * 100:.2f}%")
    print(f"Weighted Recall: {recall * 100:.2f}%")
    print(f"Weighted F1 Score: {f1 * 100:.2f}%")

In [ ]:
# dnn_model = grid_search_xgb.best_estimator_
calculate_performance_metrics(dnn_model, X_test, y_test, "original score")

In [8]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

binary_columns = find_binary_columns(X_train)

numerical_columns = []
for i in range(0,42):
    if i not in binary_columns:
        numerical_columns.append(i)

In [9]:
# xgb_base = grid_search_xgb.best_estimator_

# classifier = SklearnWrapper(dnn_model)


# rule of thumb: Interval for numerical, combination for categorical
pattern = (
    {
        "type": "interval",
        "features": numerical_columns,
        #"integer_features": list(range(10, 20)),
        "ratio": 0.1,
        "max_ratio": 0.3,
        "missing_value": 0.0,
        "probability": 1,
    },
    # {
    #     "type": "combination",
    #     "features": binary_columns,
    #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
    #     "probability": 0.4,
    # },
)

method = A2PMethod(pattern)

In [10]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [11]:
start_time = time.time()

X_adversarial = method.fit_generate(X, y)

training_time = time.time() - start_time
print(f"Attack Time: {training_time}")

ValueError: Array-like provided in 'X' must be in the (n_samples, n_features) shape.

In [ ]:
calculate_performance_metrics(dnn_model, X_adversarial, y, "full A2PM Attack")

In [ ]:

adv_samples_num = int(955009*0.10/0.9) 
adv_samples_num # 10%

In [ ]:
X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
X_adversarial_sampled = X_adversarial[X_adversarial_indices]

y_adv = np.full(adv_samples_num, 5)
y_adv

In [ ]:
X_new = X.copy()
y_new = y.copy()

X_combined = np.vstack((X_new, X_adversarial_sampled))
y_combined = np.concatenate((y_new, y_adv))

In [ ]:
X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_shape = X_train_adv.shape[1]
output_shape = len(np.unique(y_train_adv))
output_shape

In [ ]:
# model_xgb_protected = XGBClassifier()

# param_grid = {
#     # 'n_estimators': [10, 50, 100],
#     # 'max_depth': [None, 50, 100],
#     # 'n_estimators': [10],
#     # 'max_depth': [None],
# }

# grid_search_xgb_protected = GridSearchCV(model_xgb_protected, param_grid, cv=3, scoring='accuracy', verbose=3)
# grid_search_xgb_protected.fit(X_train_adv, y_train_adv)

# print("Best parameters found: ", grid_search_xgb.best_params_)
input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))


dnn_model_protected = Sequential()
dnn_model_protected.add(Input(shape=input_shape))
dnn_model_protected.add(Dense(units=30, activation='relu'))
dnn_model_protected.add(Dense(units=20, activation='relu'))
dnn_model_protected.add(Dense(units=num_classes, activation='softmax'))

opt = SGD(learning_rate=0.01)

dnn_model_protected.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
hist = dnn_model_protected.fit(X_train_adv, y_train_adv, epochs=15, batch_size=64,
                     # validation_data=(x_val, y_val),
                     callbacks=[early_stopping])

In [ ]:
# dnn_model_protected = grid_search_xgb_protected.best_estimator_

In [ ]:
X_adversarial_unseen = X_adversarial[np.logical_not(np.isin(np.arange(X_adversarial.shape[0]), X_adversarial_indices))]
len(X_adversarial_unseen)

In [ ]:
protected_adversarial_preds = dnn_model_protected.predict(X_adversarial_unseen)

In [ ]:
y_adv_full = np.full(len(X_adversarial_unseen), 5)

positive_class = 5
y_test_binary = (y_adv_full == positive_class).astype(int)
adversarial_preds_binary = (protected_adversarial_preds == positive_class).astype(int)

In [ ]:
adversarial_accuracy = accuracy_score(y_test_binary, adversarial_preds_binary)
adversarial_precision = precision_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_recall = recall_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_f1 = f1_score(y_test_binary, adversarial_preds_binary, average='binary')

print(f"Adversarial Accuracy: {adversarial_accuracy * 100:.2f}%")
print(f"Adversarial Precision: {adversarial_precision * 100:.2f}%")
print(f"Adversarial Recall: {adversarial_recall * 100:.2f}%")
print(f"Adversarial F1 Score: {adversarial_f1 * 100:.2f}%")

In [ ]:
calculate_performance_metrics(dnn_model, X_test_adv, y_test_adv, "no Adv Training")

In [ ]:
calculate_performance_metrics(dnn_model_protected, X_test_adv, y_test_adv, "w/ Adv Training")